<a href="https://colab.research.google.com/github/ayediamine/deep_learning_pg/blob/main/_colorize_black___white_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import cv2 as cv
import os.path


In [4]:
frame = cv.imread("/content/drive/MyDrive/Project_Deep_Learning/Lion.jpg")

numpy_file = np.load('/content/drive/MyDrive/Project_Deep_Learning/pts_in_hull (2).npy')

Caffe_net = cv.dnn.readNetFromCaffe("/content/drive/MyDrive/Project_Deep_Learning/The_Model/colorization_deploy_v2.prototxt", "/content/drive/MyDrive/Project_Deep_Learning/The_Model/colorization_release_v2.caffemodel")


In [5]:
numpy_file = numpy_file.transpose().reshape(2, 313, 1, 1)
Caffe_net.getLayer(Caffe_net.getLayerId('class8_ab')).blobs = [numpy_file.astype(np.float32)]
Caffe_net.getLayer(Caffe_net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]


In [6]:
input_width = 224
input_height = 224

rgb_img = (frame[:,:,[2, 1, 0]] * 1.0 / 255).astype(np.float32)
lab_img = cv.cvtColor(rgb_img, cv.COLOR_RGB2Lab)
l_channel = lab_img[:,:,0] 

l_channel_resize = cv.resize(l_channel, (input_width, input_height)) 
l_channel_resize -= 50


In [7]:
Caffe_net.setInput(cv.dnn.blobFromImage(l_channel_resize))
ab_channel = Caffe_net.forward()[0,:,:,:].transpose((1,2,0)) 

(original_height,original_width) = rgb_img.shape[:2] 
ab_channel_us = cv.resize(ab_channel, (original_width, original_height))
lab_output = np.concatenate((l_channel[:,:,np.newaxis],ab_channel_us),axis=2) 
bgr_output = np.clip(cv.cvtColor(lab_output, cv.COLOR_Lab2BGR), 0, 1)

cv.imwrite("/content/drive/MyDrive/Project_Deep_Learning/The_Model/result.png", (bgr_output*255).astype(np.uint8))


True